## This notebook loads DLC coordinates (csv) and creates plots to check tracking accuracy

*   Assumes two tracked body parts (red_led,green_led) 
*   Saves figures/percentage of coordinates above cutoff back to Google Drive

by Berkowitz & Harvey July 2020 updated by LB for ephys tracking



## Let's mount the google drive and set path to where DLC csv files are saved. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

data_path = '/content/gdrive/My Drive/DLC_analysis/Videos/'

## Alright, lets load the python packages we'll use

In [ ]:
import glob # for file names
import pandas as pd # to create data frames
from matplotlib import pyplot as plt # visualization 


## Here are some functions we'll use to automate things 

In [ ]:

# This will calculate the percent of the coordinates that are above a standard 
# cutoff (default is 95th percentile). df is created by plot_all_checks
def percent_good(df,cutoff = 0.95):
  lh = df['likelihood']
  return (np.mean(lh > cutoff))

# This is our plot function. Figure contains 4 x 1 subplots
def plot_DLC(df, cutoff = .95):
  #index out coordinates and likelihood values
  x = df['x']
  y = df['y']
  lh = df['likelihood']

  #create timestamps to align the coords over time
  ts = np.linspace(0,len(x)/30,len(x))

  #easy index for all 'good' coordinates
  x_g = x.loc[lh > cutoff]
  y_g = y.loc[lh > cutoff]
  ts_g = ts[lh > cutoff]

  #setup our figure and subplots
  fig, ax = plt.subplots(4,1,figsize=(15.0, 15.0))

  # First plot, shows all tracked coordinates
  ax[0].scatter(x,y, s = 0.05)
  ax[0].set_aspect('equal','box')
  ax[0].set_title('All Coordinates')

  # Second plot, these are only coordinates that pass cutoff
  ax[1].scatter(x_g,y_g, s = 0.05)
  ax[1].set_aspect('equal','box')
  ax[1].set_title('99th likelihood')

  # Third plot, shows x coords over time 
  ax[2].plot(ts,x, color = 'r',label = 'bad')
  ax[2].plot(ts_g,x_g,label = 'good')
  ax[2].set_title('All X coords over time')
  ax[2].legend()

  # Fourth plot, shows y coords over time 
  ax[3].plot(ts,y,color = 'r',label = 'bad')
  ax[3].plot(ts_g,y_g,label = 'good')
  ax[3].set_title('All Y coords over time')
  ax[3].legend()

  plt.tight_layout()

  return fig

# This is the main function. It reads in the csv file, computes/saves the percent
# good coords, and saves the figure back into the google drive. 
#
# Currently, the function is configured for 4 tracked body parts (head, neck,
# body, tail). 
 
def plot_all_checks(file_name):
  
  df = pd.read_csv(file_name,header = [1,2])

  df_rled = df['red_led']
  df_gled = df['green_led']
  h = []

  fig = plot_DLC(df_rled)
  h.append(percent_bad(df_rled))
  fig.savefig(file_name.replace('.csv','')+'_red_check.png')

  fig = plot_DLC(df_gled)
  h.append(percent_bad(df_gled))
  fig.savefig(file_name.replace('.csv','')+'_green_check.png')

  to_save = pd.DataFrame(h, index = ['red_led','green_led'])
  to_save.to_csv(file_name.replace('.csv','')+'_above95th.csv')


#All set! Time to run the above functions and look at your beautiful tracked coordinates :)



In [ ]:
# Find the file paths to the coordinates
files = glob.glob(data_path+'*.csv')
print(files)

# Loop through all csv coordinates in the directory, calculate/save percent good
# , and save images of the tracked body parts.
for file in range(len(files)):
  plot_all_checks(files[file])
